<a href="https://colab.research.google.com/github/ambika1us/Data_Extraction_and_Text_Analysis_for_Blackcoffer/blob/main/Ambika_Prasad_Data_Extraction_and_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files
import nltk
import string
nltk.download('punkt')
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
#importing input file
df=pd.read_excel('/content/drive/MyDrive/Blackcoffer/Input.xlsx')[['URL_ID','URL']]

In [27]:
df=df.iloc[0:150]

In [28]:
df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [21]:
#df.drop('URL_ID',axis=1,inplace=True)

<h4><b>Data extraction</b></h4>

In [30]:
#loop throgh each row in the df
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']

  # make a request to url
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #create a beautifulsoup object
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  #find title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue
  #find text
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))



can't get title of 44
can't get title of 57
can't get title of 144


In [34]:
 #write title and text to the file
file_name = '/content/drive/MyDrive/Blackcoffer/' + str(url_id) + '.txt'
with open(file_name, 'w') as file:
  file.write(title + '\n' + article)